In [1]:
import argparse
import random
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
from torch.utils.data import Dataset, DataLoader
import wandb
import sys
sys.path.append('../')
from dataset_clevr_ryan import RelationalDataset, BoundingBox
from utils import *
from tqdm.auto import tqdm

from einops import rearrange, reduce
from einops.layers.torch import Rearrange
import pickle


/viscam/projects/ns-diffusion/xingjian/miniconda3/envs/diffusion/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# pickle load from address
address2 = "/viscam/projects/ns-diffusion/ryan/LayoutGPT-master/llm_output/counting/clevr2O__gpt4-chat_layoutGPT_eval_adjusted.pkl"
address3 = "/viscam/projects/ns-diffusion/ryan/LayoutGPT-master/llm_output/counting/clevr3O__gpt4-chat_layoutGPT_eval_adjusted.pkl"
address4 = "/viscam/projects/ns-diffusion/ryan/LayoutGPT-master/llm_output/counting/clevr4O__gpt4-chat_layoutGPT_eval_adjusted.pkl"
address5 = "/viscam/projects/ns-diffusion/ryan/LayoutGPT-master/llm_output/counting/clevr5O__gpt4-chat_layoutGPT_eval_adjusted.pkl"
address8 = "/viscam/projects/ns-diffusion/ryan/LayoutGPT-master/llm_output/counting/clevr8O__gpt4-chat_layoutGPT_eval_adjusted.pkl"
result2 = pickle.load(open(address2, "rb"))
result3 = pickle.load(open(address3, "rb"))
result4 = pickle.load(open(address4, "rb"))
result5 = pickle.load(open(address5, "rb"))
result8 = pickle.load(open(address8, "rb"))

FileNotFoundError: [Errno 2] No such file or directory: '/viscam/projects/ns-diffusion/ryan/LayoutGPT-master/llm_output/counting/clevr2O__gpt4-chat_layoutGPT_eval_adjusted.pkl'

In [13]:
sys.path.append('../bbox_classifier')
from classifier import BboxClassifier
from eval_pipeline import *
metric_model = BboxClassifier()
metric_model.load_state_dict(torch.load('../bbox_classifier/4-layer-DNN-48_multi_rels-400.pth'))
def single_image_eval(bboxes, relations, relations_ids, eval_info = EvalInfo()):
    # print("entered single image eval")
    # print(f"inputs: {bboxes}, {relations}, {relations_ids}")
    correct_relations = 0
    for (i, rel) in enumerate(relations):
        (a, b) = relations_ids[i]
        a = a.item()
        b = b.item()
        # print("?", bboxes[a],bboxes[b], rel)
        rel_id = rel[-1]
        input = torch.concat([bboxes[a].cuda(), bboxes[b].cuda(), torch.tensor([rel_id]).cuda()])
        input = input.cuda()
        metric_model.cuda()
        pred = metric_model(input)[0].item()
        correct_relations += pred > 0.5
        eval_info.update(rel_id, pred > 0.5)

        # print(f"{i}th relation: {pred} vs {rel[0]}")
    return correct_relations / len(relations), eval_info

In [14]:
def evaluate(data, obj_num, wandb_drawer = None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    metric_model.to(device)
    metric_model.eval()
    all_gt_bboxes, all_gen_bboxes, all_relations, all_relation_ids = data

    size = len(all_gt_bboxes)
    assert size == 100

    eval_info = EvalInfo()
    scores = []

    all_gen_bboxes = all_gen_bboxes / 64 - 1

    for i in range(size):
        bboxes = all_gen_bboxes[i]
        relations = all_relations[i]
        relations_ids = all_relation_ids[i]
        if i == 0:
            print("bboxes", bboxes)
        score, eval_info = single_image_eval(bboxes, relations, relations_ids, eval_info)
        scores.append(score)
    
    bboxes = [[BoundingBox(e.tolist()) for e in bboxes] for bboxes in all_gen_bboxes]

    images = [None] * size
    for i in range(size):
        image = Image.new('RGB', (256, 256), (255, 255, 255))
        colours = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255), (255, 0, 255)] # red, green, blue, yellow, cyan, magenta
        for j, bbox in enumerate(bboxes[i]):
            image = bbox.draw(image, color=colours[j % len(colours)])
        images[i] = image
    if wandb_drawer is not None:
        wandb_drawer.log({"images": [wandb.Image(image) for image in images]}, step = obj_num)
    # save images to file
    for i in range(size):
        images[i].save(f"images/LayoutGPT_{obj_num}_{i}.png")

    avg_score = sum(scores) / len(scores)
    

    # separately log each relation's acc
    eval_info_list = eval_info.to_list()
    eval_info.print()
    for i in range(6):
        print(f"acc_{eval_info.relation_names[i]}: {eval_info_list[i]}")
        if wandb_drawer is not None:
            wandb_drawer.log({f"acc_{eval_info.relation_names[i]}": eval_info_list[i]}, step = obj_num)
    print(f"avg_acc: {avg_score}")
    if wandb_drawer is not None:
        wandb_drawer.log({"acc": avg_score}, step = obj_num)
    return avg_score

    

In [16]:
# wandb_drawer = None
wandb_drawer = wandb.init(
            project="diffusion_bbox_eval",
            name=f"LayoutGPT_3",
            save_code=True)
evaluate(result2, 2, wandb_drawer)
evaluate(result3, 3, wandb_drawer)
evaluate(result4, 4, wandb_drawer)
evaluate(result5, 5, wandb_drawer)
evaluate(result8, 8, wandb_drawer)
if wandb_drawer is not None:
    wandb_drawer.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xingjian-bai. Use `wandb login --relogin` to force relogin


bboxes tensor([[-0.6406,  0.0000, -0.5000, -0.4688],
        [-0.1406,  0.0000, -0.6719, -0.6562]])
left: 90 / 99
right: 90 / 99
front: 50 / 99
behind: 58 / 99
above: 0 / 1
below: 0 / 1
acc_left: 0.9090909090909091
acc_right: 0.9090909090909091
acc_front: 0.5050505050505051
acc_behind: 0.5858585858585859
acc_above: 0.0
acc_below: 0.0
avg_acc: 0.7272727272727273
bboxes tensor([[ 0.0312, -0.2344, -0.0312, -0.5781],
        [-0.5625, -0.2344, -0.7656, -0.6562],
        [-0.2812, -0.2812, -0.3125, -0.3750]])
left: 205 / 296
right: 203 / 296
front: 73 / 296
behind: 217 / 296
above: 1 / 4
below: 0 / 4
acc_left: 0.6925675675675675
acc_right: 0.6858108108108109
acc_front: 0.24662162162162163
acc_behind: 0.7331081081081081
acc_above: 0.25
acc_below: 0.0
avg_acc: 0.589527027027027
bboxes tensor([[-0.0625,  0.0781, -0.4688, -0.4062],
        [ 0.4688, -0.1562, -0.4688, -0.4844],
        [-0.6875, -0.0312, -0.4688, -0.4219],
        [ 0.3125, -0.6406, -0.5000, -0.5000]])
left: 377 / 581
right: 382

acc,█▃▄▁▁
acc_above,▁█▁▁▁
acc_behind,▃█▄▁▃
acc_below,▁▁▁▁▁
acc_front,█▁▇▄▇
acc_left,█▄▃▂▁
acc_right,█▄▃▂▁
acc,0.52213
acc_above,0.0
acc_behind,0.57764
acc_below,0.0
